在本节中，我们将从零开始实现一个基于字符级循环神经网络的语言模型，并在周杰伦专辑歌词数据集上**训练一个模型来进行歌词创作**。首先，我们读取周杰伦专辑歌词数据集：

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as f
import numpy as np
import sys
import time
sys.path.append("..") 
import d2lzh_tensorflow2 as d2l
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [2]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

## 6.4.1 one-hot向量

为了将词表示成向量输入到神经网络，一个简单的办法是使用one-hot向量。假设词典中不同字符的数量为$N$（即词典大小`vocab_size`），每个字符已经同一个从0到$N-1$的连续整数值索引一一对应。如果一个字符的索引是整数$i$, 那么我们创建一个全0的长为$N$的向量，并将其位置为$i$的元素设成1。该向量就是对原字符的one-hot向量。下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

In [2]:
tf.one_hot(np.array([0, 2]), vocab_size)

<tf.Tensor: shape=(2, 1027), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)>

我们每次采样的小批量的形状是(批量大小, 时间步数)。下面的函数将这样的小**批量变换成数个可以输入进网络的形状为(批量大小, 词典大小)的矩阵**，矩阵个数等于时间步数。也就是说，时间步$t$的输入为$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$，其中$n$为批量大小，$d$为输入个数，即one-hot向量长度（词典大小）。

In [3]:
def to_onehot(X,size):  # 本函数已保存在d2lzh_tensorflow2包中方便以后使用
    # X shape: (batch), output shape: (batch, n_class)
    return [tf.one_hot(x, size, dtype=tf.float32) for x in X.T]

X = np.arange(10).reshape((2,5))
inputs = to_onehot(X,vocab_size)
len(inputs), inputs[0].shape

(5, TensorShape([2, 1027]))

In [4]:
inputs

[<tf.Tensor: shape=(2, 1027), dtype=float32, numpy=
 array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 1027), dtype=float32, numpy=
 array([[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 1027), dtype=float32, numpy=
 array([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 1027), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 1027), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>]

## 6.4.2 初始化模型参数

接下来，我们初始化模型参数。隐藏单元个数 `num_hiddens`是一个超参数。

In [5]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def get_params():
    def _one(shape):
        return tf.Variable(tf.random.normal(shape=shape,stddev=0.01,mean=0,dtype=tf.float32))
    
    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = tf.Variable(tf.zeros(num_hiddens), dtype=tf.float32)
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = tf.Variable(tf.zeros(num_outputs), dtype=tf.float32)
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    return params

In [50]:
"""
np.random.rand(10)

np.random.randn(10)

np.random.randint((9,10))

"""

## 6.4.3 定义模型

我们根据循环神经网络的计算表达式实现该模型。首先定义`init_rnn_state`函数来返回初始化的隐藏状态。它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的`NDArray`组成的元组。使用元组是为了更便于处理隐藏状态含有多个`NDArray`的情况。

In [6]:
def init_rnn_state(batch_size, num_hiddens):
    return (tf.zeros(shape=(batch_size, num_hiddens)), )  # 这里 , 的作用，是啥？   不能删除掉

下面的`rnn`函数定义了在一个时间步里如何计算隐藏状态和输出。这里的激活函数使用了tanh函数。3.8节（多层感知机）中介绍过，当元素在实数域上均匀分布时，tanh函数值的均值为0。

In [7]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, =  state
    outputs = []
    for X in inputs:
        X = tf.reshape(X, [-1, W_xh.shape[0]])
        H = tf.tanh(tf.matmul(X, W_xh) + tf.matmul(H, W_hh) + b_h)
        Y = tf.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

做个简单的测试来观察输出结果的个数（时间步数），以及第一个时间步的输出层输出的形状和隐藏状态的形状。

In [8]:
state = init_rnn_state(X.shape[0], num_hiddens)
inputs = to_onehot(X, vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

5 (2, 1027) (2, 256)


## 6.4.4 定义预测函数

以下函数基于前缀`prefix`（含有数个字符的字符串）来预测接下来的`num_chars`个字符。这个函数**稍显复杂**，其中我们将循环神经单元`rnn`设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。

In [9]:
# 本函数已保存在d2lzh_tensorflow2包中方便以后使用
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens)   # 批次， 隐藏单元个数 
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) -1):
        # 将上一时间步的输出作为当前时间的输入
        X = tf.convert_to_tensor(to_onehot(np.array([output[-1]]),vocab_size), dtype=tf.float32)  # output[-1]  最后一个H输出
        X = tf.reshape(X, [1,-1])
        # 计算输出和更新隐藏状态
        (Y,state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符   这里有点不太理解
        if t < len(prefix) - 1:   # 填入前缀里面的词
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(np.array(tf.argmax(Y[0], axis=1))))
    #print(output)
    #print([idx_to_char[i] for i in  output])
    return ''.join([idx_to_char[i] for i in output])

我们先测试一下`predict_rnn`函数。我们将根据前缀“分开”创作长度为10个字符（不考虑前缀长度）的一段歌词。因为模型参数为随机值，所以预测结果也是随机的。

In [10]:
predict_rnn('分开', 10, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, idx_to_char, char_to_idx)

'分开钟虹病两防银笑骨有玄'

## 6.4.5 裁剪梯度

**循环神经网络中较容易出现梯度衰减或梯度爆炸**。我们会在6.6节（通过时间反向传播）中解释原因。为了应对梯度爆炸，我们可以**裁剪梯度**（clip gradient）。假设我们**把所有模型参数梯度的元素拼接成一个向量 $\boldsymbol{g}$，并设裁剪的阈值是$\theta$。裁剪后的梯度**

$$ \min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}$$

的$L_2$范数不超过$\theta$。

In [11]:
# 本函数已保存在d2lzh包中方便以后使用
# 计算裁剪后的梯度
"""
有一些没看明白
"""
def grad_clipping(grads, theta):
    norm = np.array([0])
    for i in range(len(grads)):
        norm += tf.math.reduce_sum(grads[i] ** 2)
    # print("norm",norm)
    norm = np.sqrt(norm).item()   # 这一项是 L2范数
    new_gradient = []
    if  norm > theta:   # L2范数 大于 theta
        for grad in grads:
            new_gradient.append(grad * theta / norm)   # 梯度被下降了
    else:
        for grad in grads:
            new_gradient.append(grad)
    # print("new_gradient", new_gradient)
    return new_gradient

## 6.4.6 困惑度

**我们通常使用困惑度（perplexity）来评价语言模型的好坏**。回忆一下3.4节（softmax回归）中交叉熵损失函数的定义。**困惑度是对交叉熵损失函数做指数运算后得到的值**。特别地，

* 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
* 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
* 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

显然，**任何一个有效模型的困惑度必须小于类别个数**。在本例中，困惑度必须小于词典大小`vocab_size`。

In [12]:
import math
# 本函数已保存在d2lzh包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,   # 传入 函数 等多种内容作为参数
                          vocab_size,  corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:   # 随机采样
        data_iter_fn = d2l.data_iter_random
    else:   # 相邻采样
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    #loss = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr)    
    
    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens)
        l_sum, n, start = 0.0, 0, time.time()   # l_sum是什么？   损失之和
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens)
            #else:  # 否则需要使用detach函数从计算图分离隐藏状态
                #for s in state:
                    #s.detach()
            with tf.GradientTape(persistent=True) as tape:
                    tape.watch(params)
                    inputs = to_onehot(X, vocab_size)
                    # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵，  #因为求 one_hot那里又转过一次
                    (outputs, state) = rnn(inputs, state, params)   # 获得 输出，和新的 H 值
                    # 拼接之后形状为(num_steps * batch_size, vocab_size)
                    outputs = tf.concat(outputs, 0)
                    # Y的形状是(batch_size, num_steps)，转置后再变成长度为
                    # batch * num_steps 的向量，这样跟输出的行一一对应
                    y = Y.T.reshape((-1,))      # 这里也不管其他，反正就是转成 一列 数据
                    #print(Y,y)
                    y=tf.convert_to_tensor(y,dtype=tf.float32)
                    # 使用交叉熵损失计算平均分类误差
                    l = tf.reduce_mean(tf.losses.sparse_categorical_crossentropy(y,outputs))
                    #l = loss(y,outputs)
                    #print("loss",np.array(l))
            grads = tape.gradient(l, params)
            grads=grad_clipping(grads, clipping_theta)  # 裁剪梯度
            optimizer.apply_gradients(zip(grads, params))
            #sgd(params, lr, 1 , grads)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += np.array(l).item() * len(y)
            n += len(y)
    
        if (epoch + 1) % pred_period == 0:
                print('epoch %d, perplexity %f, time %.2f sec' % (
                    epoch + 1, math.exp(l_sum / n), time.time() - start))
                #print(params)
                for prefix in prefixes:
                    print(prefix)
                    print(' -', predict_rnn(
                        prefix, pred_len, rnn, params, init_rnn_state,
                        num_hiddens, vocab_size,  idx_to_char, char_to_idx))

## 6.4.8 训练模型并创作歌词

现在我们可以训练模型了。首先，设置模型超参数。**我们将根据前缀“分开”和“不分开”分别创作长度为50个字符（不考虑前缀长度）的一段歌词**。我们**每过50个迭代周期**便**根据当前训练的模型创作一段歌词**。

In [13]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

下面采用随机采样训练模型并创作歌词。

In [14]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 94.562245, time 1.02 sec
分开
 - 分开                                                  
不分开
 - 不分开                                                  
epoch 100, perplexity 21.831246, time 0.82 sec
分开
 - 分开                                                  
不分开
 - 不分开                                                  
epoch 150, perplexity 355.820099, time 0.73 sec
分开
 - 分开你你的猪你你你你 我你你你 我听你你你 我你你你你 我你你你 我听你你你 我你你你你 我你你你 我听
不分开
 - 不分开不了你阻止一直在你们你遇见你你一场悲剧你你你种 你你你你 我你你你 我听你你你爱女人你坏坏的让我 你
epoch 200, perplexity 6528.831678, time 0.57 sec
分开
 - 分开愁愁翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻
不分开
 - 不分开愁翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻翻
epoch 250, perplexity 994.155957, time 0.57 sec
分开
 - 分开爽髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅
不分开
 - 不分开髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅髅


**什么鬼。。。**

接下来采用相邻采样训练模型并创作歌词。

In [15]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 80.104102, time 0.88 sec
分开
 - 分开                                                  
不分开
 - 不分开                                                  
epoch 100, perplexity 23.879459, time 0.62 sec
分开
 - 分开 我 我   我 我 我 我 我 我 我有 我有 我     我有 我有 我有 我有 我有 我有 我
不分开
 - 不分开 我 我 我 我 我 我 我 我 我 我有  我 我有 我   我 我有 你   我 我   我有 
epoch 150, perplexity 4140.843154, time 0.63 sec
分开
 - 分开始碰蜥同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同
不分开
 - 不分开者陪同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌同涌
epoch 200, perplexity 348008.847385, time 0.55 sec
分开
 - 分开棍棍啊站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼
不分开
 - 不分开棍啊站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠站彿呼宠
epoch 250, perplexity 1013.577894, time 0.60 sec
分开
 - 分开密密令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令
不分开
 - 不分开搞令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令令


## 小结

* 可以用基于字符级循环神经网络的语言模型来生成文本序列，例如创作歌词。
* 当训练循环神经网络时，为了应对梯度爆炸，可以裁剪梯度。
* **困惑度是对交叉熵损失函数做指数运算后得到的值**。



-----------

> 注：除代码外本节与原书此节基本相同，[原书传送门](https://zh.d2l.ai/chapter_recurrent-neural-networks/rnn-scratch.html)

